In [ ]:
import os
import sys
sys.path.append("../..")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dtaidistance import dtw
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

from src.models.vaeconv import *
from src.train.trainer.TrainerVAE import Trainer
from src.data.preprocessing.pipeline import Pipeline
from src.data.datasets.universal_dataset import CVADataset
from src.data.preprocessing.splitter import select_test_inh
from src.utils.paths import get_project_path

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
INHIBITOR_NAME = "2-mercaptobenzimidazole"

prep = Pipeline(
    num_cycle=[1, 2, 3, 4], 
    inhibitor_name="all", 
    split="all",
    norm_feat=True
)
data = prep.full_data


train, val = select_test_inh(data, INHIBITOR_NAME)

In [ ]:
vae = VAE_CONV(
    in_channels=1,
    latent_dim=64,
    seq_len=968,
    desc_dim=41
).to(device)

vae.load_state_dict(
    torch.load(
        os.path.join(get_project_path(), "models", "vae_conv", INHIBITOR_NAME, "best_model.pt"),
        map_location="cpu"
    )
)

In [ ]:
full_ds = CVADataset(val)
infer_loader = DataLoader(full_ds, batch_size=1, shuffle=False)

gen_vah, real_cva = [], []

vae.eval()
with torch.inference_mode():
    for batch in infer_loader:
        vah, desc = batch["vah"].to(device), batch["features"].to(device)
        vah_hat, _, _ = vae(vah, desc)
        
        gen_vah.append(vah_hat.squeeze().detach().cpu().numpy())
        real_cva.append(vah.detach().cpu().numpy()[0])

df_vah = pd.DataFrame(gen_vah)
df_vah["Inhibitor"] = val.reset_index(drop=True)["Inhibitor"]

real_df_vah = pd.DataFrame(real_cva)
real_df_vah["Inhibitor"] = val.reset_index(drop=True)["Inhibitor"]

In [ ]:
def compute_dtw(original, reconstructed):
    """
    Dynamic Time Warping расстояние между временными рядами.
    """
    return dtw.distance_fast(original, reconstructed, use_c = True)

In [ ]:
np_vah = np.array(df_vah.drop(columns=["Inhibitor"]), dtype=np.float64)
np_valid = np.array(real_df_vah.drop(columns=["Inhibitor"]), dtype=np.float64)

In [ ]:
dtw_m = []

for i in tqdm(range(len(df_vah))):
    dtw_m.append(compute_dtw(np_vah[i], np_valid[i]))

In [ ]:
metr = pd.DataFrame(
    {
        "Inhibitor": val.reset_index(drop=True)["Inhibitor"],
        "metrics":dtw_m
    }
)

In [ ]:
n = 459

plt.figure(figsize=(10, 5))
plt.plot(np_valid[n], label="Original")
plt.plot(np_vah[n], label="Reconstructed", linestyle='--', color='red')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel("Time, s", fontsize=16, fontweight='bold', labelpad=20)
plt.ylabel("Current, A", fontsize=16, fontweight='bold', labelpad=20)
plt.legend(fontsize=14, )
plt.show()

In [ ]:
pd.DataFrame(
    {
        "Orig": np_valid[n],
        "Reconstructed": np_vah[n]
    }
).to_excel("CVA_conv_vae.xlsx", index=False)